Inspeccionar el dataset

In [ ]:
print(df.info())
print(df.isnull().sum())

Manejo de valores nulos

In [ ]:
df.dropna(inplace=True)

Convertir Columnas de Fecha y Hora a Características Numéricas

In [ ]:
# Convertir la columna 'Fecha y Hora de Llegada' a datetime
df['Fecha y Hora de Llegada'] = pd.to_datetime(df['Fecha y Hora de Llegada'])

# Extraer características
df['Year'] = df['Fecha y Hora de Llegada'].dt.year
df['Month'] = df['Fecha y Hora de Llegada'].dt.month
df['Day'] = df['Fecha y Hora de Llegada'].dt.day

Codificación de Variables Categóricas

In [ ]:
# Codificación de Variables Categóricas
df_encoded = pd.get_dummies(df, columns=['Punto de Destino', 'Punto de Salida', 'Tipo de Zona', 'Día de la Semana', 'Barrio'], drop_first=True)

División del dataset

In [ ]:
X = df_encoded.drop(columns=['He Aparcado (Si/No)'])  # Variables independientes
y = df_encoded['He Aparcado (Si/No)']  # Variable dependiente

Separar en Conjuntos de Entrenamiento y Prueba

In [ ]:
from sklearn.model_selection import train_test_split

# División en conjunto de entrenamiento y prueba (80-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Definir la Función de Optuna para Optimización de Hiperparámetros

In [ ]:
import optuna
from xgboost import XGBClassifier  # O XGBRegressor si estás usando regresión
from sklearn.metrics import accuracy_score

def objective(trial):
    # Definir los hiperparámetros a optimizar
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'use_label_encoder': False,
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
    }

    # Entrenar el modelo
    model = XGBClassifier(**params)
    model.fit(X_train, y_train)

    # Hacer predicciones en el conjunto de prueba
    y_pred = model.predict(X_test)
    
    # Calcular la precisión
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy


Crear un Estudio de Optuna y Optimizar

In [ ]:
# Crear un estudio y optimizar
study = optuna.create_study(direction='maximize')  # Maximizamos la precisión
study.optimize(objective, n_trials=100)  # Número de pruebas

Entrenamiento final con los mejores hiperparámetros

In [ ]:
# Imprimir los mejores hiperparámetros
print("Mejores hiperparámetros:")
print(study.best_params)

# Entrenamiento final con los mejores hiperparámetros
best_params = study.best_params
final_model = XGBClassifier(**best_params)
final_model.fit(X_train, y_train)


Predicción

In [ ]:
# Predicciones finales
final_y_pred = final_model.predict(X_test)

# Evaluación del Modelo final
final_accuracy = accuracy_score(y_test, final_y_pred)
print(f'Accuracy final: {final_accuracy:.2f}')
